In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

!mkdir /root/.kaggle
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle/kaggle.json
!kaggle datasets download -d hugomathien/soccer
!unzip /content/soccer.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 86% 28.0M/32.7M [00:00<00:00, 52.3MB/s]
100% 32.7M/32.7M [00:00<00:00, 60.6MB/s]
Archive:  /content/soccer.zip
  inflating: database.sqlite         


In [ ]:
## Importing required libraries
import sqlite3
import pandas as pd
import numpy as np
import itertools
from time import time
import warnings
import os

warnings.simplefilter("ignore")

In [ ]:
start = time()
## Fetching data
#Connecting to database
path = "/content/"  #Insert path here
database = path + 'database.sqlite'
conn = sqlite3.connect(database)

#Fetching required data tables
player_data = pd.read_sql("SELECT * FROM Player;", conn)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", conn)
team_data = pd.read_sql("SELECT * FROM Team;", conn)
match_data = pd.read_sql("SELECT * FROM Match;", conn)
league_data = pd.read_sql("SELECT * FROM League;", conn)
team_attributes_data = pd.read_sql("SELECT * FROM Team_Attributes", conn)

rows = ["country_id", "league_id", "season", "stage", "date", "match_api_id", "home_team_api_id", 
        "away_team_api_id", "home_team_goal", "away_team_goal", "home_player_1", "home_player_2",
        "home_player_3", "home_player_4", "home_player_5", "home_player_6", "home_player_7", 
        "home_player_8", "home_player_9", "home_player_10", "home_player_11", "away_player_1",
        "away_player_2", "away_player_3", "away_player_4", "away_player_5", "away_player_6",
        "away_player_7", "away_player_8", "away_player_9", "away_player_10", "away_player_11"]
match_data.dropna(subset = rows, inplace = True)

# match_data = match_data.tail(1500) # comment away this at the end to get full data

In [ ]:
def tables_in_sqlite_db(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor.close()
    return tables

tables = tables_in_sqlite_db(conn)
tables

[]

In [ ]:
# only retrieve the latest attributes for each team
team_stats_new = pd.DataFrame()
#Loop through all teams
for team_api_id in team_data.team_api_id.unique():   
    #Get team stats 
    stats = team_attributes_data[team_attributes_data.team_api_id == team_api_id]
  
    #Identify current stats       
    current_stats = stats.sort_values(by = 'date', ascending = False)[:1]
    current_stats.reset_index(inplace = True, drop = True)
       
    #Aggregate stats
    team_stats_new = pd.concat([team_stats_new, current_stats], axis = 0)

team_stats_new.reset_index(inplace = True, drop = True)
team_attributes_data = team_stats_new
del team_stats_new

In [ ]:
# Use the following attributes in team_attributes table for match outcome prediction

team_data = pd.merge(team_data, team_attributes_data[['team_api_id', 'buildUpPlaySpeedClass', 
                                                    'chanceCreationPassingClass', 'defenceDefenderLineClass']],
                   how='inner', on='team_api_id')

In [ ]:
def get_fifa_stats(match, player_stats):
    ''' Aggregates fifa stats for a given match. '''    
    
    #Define variables
    match_id =  match.match_api_id
    date = match['date']
    players = ['home_player_1', 'home_player_2', 'home_player_3', "home_player_4", "home_player_5",
               "home_player_6", "home_player_7", "home_player_8", "home_player_9", "home_player_10",
               "home_player_11", "away_player_1", "away_player_2", "away_player_3", "away_player_4",
               "away_player_5", "away_player_6", "away_player_7", "away_player_8", "away_player_9",
               "away_player_10", "away_player_11"]
    player_stats_new = pd.DataFrame()
    names = []
    
    #Loop through all players
    for player in players:   
            
        #Get player ID
        player_id = match[player]
        
        #Get player stats 
        stats = player_stats[player_stats.player_api_id == player_id]
            
        #Identify current stats       
        current_stats = stats[stats.date < date].sort_values(by = 'date', ascending = False)[:1]
        
        if np.isnan(player_id) == True:
            overall_rating = pd.Series(0)
        else:
            current_stats.reset_index(inplace = True, drop = True)
            overall_rating = pd.Series(current_stats.loc[0, "overall_rating"])

        #Rename stat
        name = "{}_overall_rating".format(player)
        names.append(name)
            
        #Aggregate stats
        player_stats_new = pd.concat([player_stats_new, overall_rating], axis = 1)
    
    player_stats_new.columns = names        
    player_stats_new['match_api_id'] = match_id

    player_stats_new.reset_index(inplace = True, drop = True)
    
    #Return player stats    
    return player_stats_new.loc[0]   

def get_fifa_data(matches, player_stats, path = None, data_exists = False):
    ''' Gets fifa data for all matches. '''  
    
    #Check if fifa data already exists
    if data_exists == True:
        
        fifa_data = pd.read_pickle(path)
        
    else:
        
        print("Collecting fifa data for each match...")       
        start = time()
        
        #Apply get_fifa_stats for each match
        fifa_data = matches.apply(lambda x :get_fifa_stats(x, player_stats), axis = 1)
        
        end = time()    
        print("Fifa data collected in {:.1f} minutes".format((end - start)/60))
    
    #Return fifa_data
    return fifa_data
## Generating features, exploring the data, and preparing data for model training
#Generating or retrieving already existant FIFA data
fifa_data_path = "/content/drive/MyDrive/School/BC 2407/dataset/fifa_data.pkl"
fifa_data = get_fifa_data(match_data, player_stats_data, path = fifa_data_path, data_exists = os.path.isfile(fifa_data_path))


In [ ]:
import pickle
with open("/content/drive/MyDrive/School/BC 2407/dataset/fifa_data.pkl", "wb") as f:
  pickle.dump(fifa_data, f)

In [ ]:
def get_match_label(match):
    ''' Derives a label for a given match. '''
    
    #Define variables
    home_goals = match['home_team_goal']
    away_goals = match['away_team_goal']
     
    label = pd.DataFrame()
    label.loc[0,'match_api_id'] = match['match_api_id'] 

    #Identify match label  
    if home_goals > away_goals:
        label.loc[0,'label'] = "Win"
    if home_goals == away_goals:
        label.loc[0,'label'] = "Draw"
    if home_goals < away_goals:
        label.loc[0,'label'] = "Defeat"

    #Return label        
    return label.loc[0]

def get_wins(matches, team):
    ''' Get the number of wins of a specfic team from a set of matches. '''
    
    #Find home and away wins
    home_wins = int(matches.home_team_goal[(matches.home_team_api_id == team) & (matches.home_team_goal > matches.away_team_goal)].count())
    away_wins = int(matches.away_team_goal[(matches.away_team_api_id == team) & (matches.away_team_goal > matches.home_team_goal)].count())

    total_wins = home_wins + away_wins

    #Return total wins
    return total_wins      

def get_goals_conceided(matches, team):
    ''' Get the goals conceided of a specfic team from a set of matches. '''

    #Find home and away goals
    home_goals = int(matches.home_team_goal[matches.away_team_api_id == team].sum())
    away_goals = int(matches.away_team_goal[matches.home_team_api_id == team].sum())

    total_goals = home_goals + away_goals

    #Return total goals
    return total_goals

def get_goals(matches, team):
    ''' Get the goals of a specfic team from a set of matches. '''
    
    #Find home and away goals
    home_goals = int(matches.home_team_goal[matches.home_team_api_id == team].sum())
    away_goals = int(matches.away_team_goal[matches.away_team_api_id == team].sum())

    total_goals = home_goals + away_goals
    
    #Return total goals
    return total_goals

def get_last_matches(matches, date, team, x = 10):
    ''' Get the last x matches of a given team. '''
    
    #Filter team matches from matches
    team_matches = matches[(matches['home_team_api_id'] == team) | (matches['away_team_api_id'] == team)]
                           
    #Filter x last matches from team matches
    last_matches = team_matches[team_matches.date < date].sort_values(by = 'date', ascending = False).iloc[0:x,:]
    
    #Return last matches
    return last_matches

def get_last_matches_against_eachother(matches, date, home_team, away_team, x = 10):
    ''' Get the last x matches of two given teams. '''
    
    #Find matches of both teams
    home_matches = matches[(matches['home_team_api_id'] == home_team) & (matches['away_team_api_id'] == away_team)]    
    away_matches = matches[(matches['home_team_api_id'] == away_team) & (matches['away_team_api_id'] == home_team)]  
    total_matches = pd.concat([home_matches, away_matches])
    
    #Get last x matches
    try:    
        last_matches = total_matches[total_matches.date < date].sort_values(by = 'date', ascending = False).iloc[0:x,:]
    except:
        last_matches = total_matches[total_matches.date < date].sort_values(by = 'date', ascending = False).iloc[0:total_matches.shape[0],:]
        
        #Check for error in data
        if(last_matches.shape[0] > x):
            print("Error in obtaining matches")
            
    #Return data
    return last_matches

def get_match_features(match, matches, x = 10):
    ''' Create match specific features for a given match. '''
    
    #Define variables
    date = match.date
    home_team = match.home_team_api_id
    away_team = match.away_team_api_id
    
    #Get last x matches of home and away team
    matches_home_team = get_last_matches(matches, date, home_team, x = 10)
    matches_away_team = get_last_matches(matches, date, away_team, x = 10)
    
    #Get last x matches of both teams against each other
    last_matches_against = get_last_matches_against_eachother(matches, date, home_team, away_team, x = 5)
    
    #Create goal variables
    home_goals = get_goals(matches_home_team, home_team)
    away_goals = get_goals(matches_away_team, away_team)
    home_goals_conceided = get_goals_conceided(matches_home_team, home_team)
    away_goals_conceided = get_goals_conceided(matches_away_team, away_team)
    
    #Define result data frame
    result = pd.DataFrame()
    
    #Define ID features
    result.loc[0, 'match_api_id'] = match.match_api_id
    result.loc[0, 'league_id'] = match.league_id

    #Create match features
    result.loc[0, 'home_team_goals_difference'] = home_goals - home_goals_conceided
    result.loc[0, 'away_team_goals_difference'] = away_goals - away_goals_conceided
    result.loc[0, 'games_won_home_team'] = get_wins(matches_home_team, home_team) 
    result.loc[0, 'games_won_away_team'] = get_wins(matches_away_team, away_team)
    result.loc[0, 'games_against_won'] = get_wins(last_matches_against, home_team)
    result.loc[0, 'games_against_lost'] = get_wins(last_matches_against, away_team)
     
    #Return match features
    return result.loc[0]

def get_overall_fifa_rankings(fifa, get_overall = False):
    ''' Get overall fifa rankings from fifa data. '''
      
    temp_data = fifa
    
    #Check if only overall player stats are desired
    if get_overall == True:
        
        #Get overall stats
        data = temp_data.loc[:,(fifa.columns.str.contains('overall_rating'))]
        data.loc[:,'match_api_id'] = temp_data.loc[:,'match_api_id']
    else:
        
        #Get all stats except for stat date
        cols = fifa.loc[:,(fifa.columns.str.contains('date_stat'))]
        temp_data = fifa.drop(cols.columns, axis = 1)        
        data = temp_data
    
    #Return data
    return data

def create_feables(matches, fifa, bookkeepers, get_overall = False, horizontal = True, x = 10, verbose = True):
    ''' Create and aggregate features and labels for all matches. '''

    #Get fifa stats features
    fifa_stats = get_overall_fifa_rankings(fifa, get_overall)
    
    
    if verbose == True:
        print("Generating match features...")
    start = time()
    
    #Get match features for all matches
    match_stats = matches.apply(lambda x: get_match_features(x, matches, x = 10), axis = 1)
    
    #Create dummies for league ID feature
    # dummies = pd.get_dummies(match_stats['league_id']).rename(columns = lambda x: 'League_' + str(x))
    # match_stats = pd.concat([match_stats, dummies], axis = 1)
    # match_stats.drop(['league_id'], inplace = True, axis = 1)
    
    end = time()
    if verbose == True:
        print("Match features generated in {:.1f} minutes".format((end - start)/60))
    
    if verbose == True:    
        print("Generating match labels...")
    start = time()
    
    #Create match labels
    labels = matches.apply(get_match_label, axis = 1)
    end = time()
    if verbose == True:
        print("Match labels generated in {:.1f} minutes".format((end - start)/60))
    
    if verbose == True:    
        print("Generating bookkeeper data...")
    start = time()
    
    #Get bookkeeper quotas for all matches
    bk_data = get_bookkeeper_data(matches, bookkeepers, horizontal = True)
    bk_data.loc[:,'match_api_id'] = matches.loc[:,'match_api_id']
    end = time()
    if verbose == True:
        print("Bookkeeper data generated in {:.1f} minutes".format((end - start)/60))

    #Merges features and labels into one frame
    features = pd.merge(match_stats, fifa_stats, on = 'match_api_id', how = 'left')
    features = pd.merge(features, bk_data, on = 'match_api_id', how = 'left')
    feables = pd.merge(features, labels, on = 'match_api_id', how = 'left')
    
    #Drop NA values
    feables.dropna(inplace = True)
    
    #Return preprocessed data
    return feables

def convert_odds_to_prob(match_odds):
    ''' Converts bookkeeper odds to probabilities. '''
    
    #Define variables
    match_id = match_odds.loc[:,'match_api_id']
    bookkeeper = match_odds.loc[:,'bookkeeper']    
    win_odd = match_odds.loc[:,'Win']
    draw_odd = match_odds.loc[:,'Draw']
    loss_odd = match_odds.loc[:,'Defeat']
    
    #Converts odds to prob
    win_prob = 1 / win_odd
    draw_prob = 1 / draw_odd
    loss_prob = 1 / loss_odd
    
    total_prob = win_prob + draw_prob + loss_prob
    
    probs = pd.DataFrame()
    
    #Define output format and scale probs by sum over all probs
    probs.loc[:,'match_api_id'] = match_id
    probs.loc[:,'bookkeeper'] = bookkeeper
    probs.loc[:,'Win'] = win_prob / total_prob
    probs.loc[:,'Draw'] = draw_prob / total_prob
    probs.loc[:,'Defeat'] = loss_prob / total_prob
    
    #Return probs and meta data
    return probs

def get_bookkeeper_data(matches, bookkeepers, horizontal = True):
    ''' Aggregates bookkeeper data for all matches and bookkeepers. '''
    
    bk_data = pd.DataFrame()
    
    #Loop through bookkeepers
    for bookkeeper in bookkeepers:

        #Find columns containing data of bookkeeper
        temp_data = matches.loc[:,(matches.columns.str.contains(bookkeeper))]
        temp_data.loc[:, 'bookkeeper'] = str(bookkeeper)
        temp_data.loc[:, 'match_api_id'] = matches.loc[:, 'match_api_id']
        
        #Rename odds columns and convert to numeric
        cols = temp_data.columns.values
        cols[:3] = ['Win','Draw','Defeat']
        temp_data.columns = cols
        temp_data.loc[:,'Win'] = pd.to_numeric(temp_data['Win'])
        temp_data.loc[:,'Draw'] = pd.to_numeric(temp_data['Draw'])
        temp_data.loc[:,'Defeat'] = pd.to_numeric(temp_data['Defeat'])
        
        #Check if data should be aggregated horizontally
        if(horizontal == True):
            
            #Convert data to probs
            temp_data = convert_odds_to_prob(temp_data)
            temp_data.drop('match_api_id', axis = 1, inplace = True)
            temp_data.drop('bookkeeper', axis = 1, inplace = True)
            
            #Rename columns with bookkeeper names
            win_name = bookkeeper + "_" + "Win"
            draw_name = bookkeeper + "_" + "Draw"
            defeat_name = bookkeeper + "_" + "Defeat"
            temp_data.columns.values[:3] = [win_name, draw_name, defeat_name]

            #Aggregate data
            bk_data = pd.concat([bk_data, temp_data], axis = 1)
        else:
            #Aggregate vertically
            bk_data = bk_data.append(temp_data, ignore_index = True)
    
    #If horizontal add match api id to data
    if(horizontal == True):
        temp_data.loc[:, 'match_api_id'] = matches.loc[:, 'match_api_id']
    
    #Return bookkeeper data
    return bk_data
    
def get_bookkeeper_probs(matches, bookkeepers, horizontal = False):
    ''' Get bookkeeper data and convert to probabilities for vertical aggregation. '''
    
    #Get bookkeeper data
    data = get_bookkeeper_data(matches, bookkeepers, horizontal = False)
    
    #Convert odds to probabilities
    probs = convert_odds_to_prob(data)
    
    #Return data
    return probs


In [ ]:
# Creating features and labels based on data provided
bk_cols = ['B365', 'BW', 'IW', 'LB', 'PS', 'WH', 'SJ', 'VC', 'GB', 'BS']
bk_cols_selected = ['B365', 'BW']      
feables = create_feables(match_data, fifa_data, bk_cols_selected, get_overall = True)

# getting team overall ratings
home_team_rating_cols = ['home_player_' + str(i) + "_overall_rating" for i in range(1, 12)]
away_team_rating_cols = ['away_player_' + str(i) + "_overall_rating" for i in range(1, 12)]
feables['home_team_overall_rating'], feables['away_team_overall_rating'] = 0, 0
for home, away in zip(home_team_rating_cols, away_team_rating_cols):
  feables['home_team_overall_rating'] += feables[home]
  feables['away_team_overall_rating'] += feables[away]
  feables = feables.drop(home, axis=1)
  feables = feables.drop(away, axis=1)

# getting league names
temp_df = league_data.copy()
temp_df['league_id'] = temp_df['id']
temp_df['league_name'] = temp_df['name']
temp_df.drop(['country_id', 'name', 'id'], axis=1, inplace=True)
feables = pd.merge(feables, temp_df, how='inner', on='league_id')
feables.drop('league_id', axis=1, inplace=True)
del temp_df

# get home and away team names
temp_df = pd.merge(match_data[['match_api_id', 'home_team_api_id', 'away_team_api_id']], 
         team_data[['team_api_id', 'team_long_name', 'buildUpPlaySpeedClass',
                    'chanceCreationPassingClass', 'defenceDefenderLineClass']], 
         how='inner', left_on='home_team_api_id', right_on='team_api_id')

temp_df.rename(columns={
    'team_long_name': 'home_team_long_name',
    'buildUpPlaySpeedClass': 'home_team_buildUpPlaySpeedClass',
    'chanceCreationPassingClass': 'home_team_chanceCreationPassingClass',
    'defenceDefenderLineClass': 'home_team_defenceDefenderLineClass'
}, inplace=True)
temp_df.drop(['team_api_id', 'home_team_api_id'], axis=1, inplace=True)

temp_df = pd.merge(temp_df, 
         team_data[['team_api_id', 'team_long_name', 'buildUpPlaySpeedClass',
                    'chanceCreationPassingClass', 'defenceDefenderLineClass']], 
         how='inner', left_on='away_team_api_id', right_on='team_api_id')
temp_df.rename(columns={
    'team_long_name': 'away_team_long_name',
    'buildUpPlaySpeedClass': 'away_team_buildUpPlaySpeedClass',
    'chanceCreationPassingClass': 'away_team_chanceCreationPassingClass',
    'defenceDefenderLineClass': 'away_team_defenceDefenderLineClass'
}, inplace=True)

temp_df.drop(['team_api_id', 'away_team_api_id'], axis=1, inplace=True)

feables = pd.merge(feables, temp_df, how='inner', on='match_api_id')
del temp_df
feables.head()

Generating match features...
Match features generated in 10.5 minutes
Generating match labels...
Match labels generated in 1.1 minutes
Generating bookkeeper data...
Bookkeeper data generated in 0.0 minutes


,match_api_id,home_team_goals_difference,away_team_goals_difference,games_won_home_team,games_won_away_team,games_against_won,games_against_lost,B365_Win,B365_Draw,B365_Defeat,BW_Win,BW_Draw,BW_Defeat,label,home_team_overall_rating,away_team_overall_rating,league_name,home_team_long_name,home_team_buildUpPlaySpeedClass,home_team_chanceCreationPassingClass,home_team_defenceDefenderLineClass,away_team_long_name,away_team_buildUpPlaySpeedClass,away_team_chanceCreationPassingClass,away_team_defenceDefenderLineClass
0,493017.0,0.0,0.0,0.0,0.0,0.0,0.0,0.313804,0.276886,0.409310,0.307825,0.279410,0.412765,Win,657.0,705.0,Belgium Jupiler League,KV Mechelen,Balanced,Normal,Cover,KRC Genk,Balanced,Safe,Cover
1,493025.0,0.0,0.0,0.0,0.0,0.0,0.0,0.327179,0.286281,0.386540,0.290493,0.300176,0.409331,Defeat,724.0,754.0,Belgium Jupiler League,KSV Cercle Brugge,Balanced,Normal,Cover,Club Brugge KV,Balanced,Normal,Cover
2,493027.0,0.0,0.0,0.0,0.0,0.0,0.0,0.672897,0.209346,0.117757,0.672269,0.226891,0.100840,Win,775.0,671.0,Belgium Jupiler League,RSC Anderlecht,Balanced,Normal,Cover,SV Zulte-Waregem,Balanced,Normal,Cover
3,493034.0,1.0,2.0,1.0,1.0,0.0,0.0,0.207407,0.259259,0.533333,0.192717,0.274476,0.532807,Win,658.0,776.0,Belgium Jupiler League,KV Mechelen,Balanced,Normal,Cover,RSC Anderlecht,Balanced,Normal,Cover
4,493040.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.535211,0.267606,0.197183,0.565759,0.254990,0.179250,Draw,683.0,660.0,Belgium Jupiler League,SV Zulte-Waregem,Balanced,Normal,Cover,KSV Roeselare,Balanced,Normal,Cover


In [ ]:
formation_cols = ['match_api_id'] +\
                 ['home_player_Y' + str(i) for i in range(1, 12)] +\
                 ['away_player_Y' + str(i) for i in range(1, 12)]
                 
formation_df = match_data[match_data['match_api_id'].isin(list(feables['match_api_id']))][formation_cols]
formation_df.dropna(inplace=True)
formation_df.astype(int)
formation_df.head()

,match_api_id,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11
145,493017,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0
153,493025,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0
155,493027,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0
162,493034,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0
168,493040,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0


In [ ]:
def get_formation(row):
  # given a Series of player's Y position for all 11 players
  # return the team formation as a string (e.g. "4-2-3-1")
  # row is a length 11 Series
  assert (len(row)) == 11
  formation = {}
  for y_pos in row[1:]: # start from the second element as the first element is assumed to always be the goalkeeper
    try:
      formation[y_pos] += 1
    except:
      formation[y_pos] = 1
  output_str = ""
  for k, v in formation.items():
      output_str += str(v) + "-"

  return output_str[:-1]

get_formation([1,3,3,3,3,6,6,8,8,8,11])

'4-2-3-1'

In [ ]:
formation_df['home_formation'] = formation_df.apply(lambda row: get_formation(row[1:12]), axis=1)
formation_df['away_formation'] = formation_df.apply(lambda row: get_formation(row[12:23]), axis=1)
formation_df.drop(['home_player_Y' + str(i) for i in range(1, 12)] +\
                 ['away_player_Y' + str(i) for i in range(1, 12)], axis=1, inplace=True)
formation_df.head()

,match_api_id,home_formation,away_formation
145,493017,4-4-2,4-4-2
153,493025,4-4-2,4-4-2
155,493027,4-4-2,4-4-2
162,493034,4-4-2,4-4-2
168,493040,4-4-2,4-4-2


In [ ]:
feables = pd.merge(feables, formation_df, how='inner', on='match_api_id')
print(len(feables.axes[0]))
feables.head()

19585


,match_api_id,home_team_goals_difference,away_team_goals_difference,games_won_home_team,games_won_away_team,games_against_won,games_against_lost,B365_Win,B365_Draw,B365_Defeat,BW_Win,BW_Draw,BW_Defeat,label,home_team_overall_rating,away_team_overall_rating,league_name,home_team_long_name,home_team_buildUpPlaySpeedClass,home_team_chanceCreationPassingClass,home_team_defenceDefenderLineClass,away_team_long_name,away_team_buildUpPlaySpeedClass,away_team_chanceCreationPassingClass,away_team_defenceDefenderLineClass,home_formation,away_formation
0,493017.0,0.0,0.0,0.0,0.0,0.0,0.0,0.313804,0.276886,0.409310,0.307825,0.279410,0.412765,Win,657.0,705.0,Belgium Jupiler League,KV Mechelen,Balanced,Normal,Cover,KRC Genk,Balanced,Safe,Cover,4-4-2,4-4-2
1,493025.0,0.0,0.0,0.0,0.0,0.0,0.0,0.327179,0.286281,0.386540,0.290493,0.300176,0.409331,Defeat,724.0,754.0,Belgium Jupiler League,KSV Cercle Brugge,Balanced,Normal,Cover,Club Brugge KV,Balanced,Normal,Cover,4-4-2,4-4-2
2,493027.0,0.0,0.0,0.0,0.0,0.0,0.0,0.672897,0.209346,0.117757,0.672269,0.226891,0.100840,Win,775.0,671.0,Belgium Jupiler League,RSC Anderlecht,Balanced,Normal,Cover,SV Zulte-Waregem,Balanced,Normal,Cover,4-4-2,4-4-2
3,493034.0,1.0,2.0,1.0,1.0,0.0,0.0,0.207407,0.259259,0.533333,0.192717,0.274476,0.532807,Win,658.0,776.0,Belgium Jupiler League,KV Mechelen,Balanced,Normal,Cover,RSC Anderlecht,Balanced,Normal,Cover,4-4-2,4-4-2
4,493040.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.535211,0.267606,0.197183,0.565759,0.254990,0.179250,Draw,683.0,660.0,Belgium Jupiler League,SV Zulte-Waregem,Balanced,Normal,Cover,KSV Roeselare,Balanced,Normal,Cover,4-4-2,4-4-2


In [ ]:
feables.to_csv("feables_with_team_attributes.csv", index=False)

In [ ]:
# getting the latest list of players in each team
latest_team_players = pd.DataFrame()
for team in team_data.team_api_id.unique():
  temp_df = match_data[(match_data['home_team_api_id'] == team) | (match_data['away_team_api_id'] == team)]
  latest_temp_df = temp_df.sort_values(by = 'date', ascending = False)[:1]
  latest_temp_df.reset_index(inplace = True, drop = True)
  try:
    team_place = 'home' if latest_temp_df['home_team_api_id'].values[0] == team else 'away'
    cols = [team_place+'_team_api_id']+[team_place+'_player_' + str(i+1) for i in range(11)]
    latest_temp_df = latest_temp_df[cols]
    latest_temp_df.rename(columns={
        team_place+'_team_api_id': 'team_name',
        team_place+'_player_1': 'player_1',
        team_place+'_player_2': 'player_2',
        team_place+'_player_3': 'player_3',
        team_place+'_player_4': 'player_4',
        team_place+'_player_5': 'player_5',
        team_place+'_player_6': 'player_6',
        team_place+'_player_7': 'player_7',
        team_place+'_player_8': 'player_8',
        team_place+'_player_9': 'player_9',
        team_place+'_player_10': 'player_10',
        team_place+'_player_11': 'player_11',
    }, inplace=True)
    latest_temp_df['team_name'][0] = team_data[team_data['team_api_id'] == latest_temp_df['team_name'][0]]['team_long_name'].values[0]
    for i in range(11):
      latest_temp_df['player_'+str(i+1)][0] = player_data[player_data['player_api_id'] == latest_temp_df['player_'+str(i+1)][0]]['player_name'].values[0]
    latest_team_players = pd.concat([latest_team_players, latest_temp_df], axis = 0)
  except IndexError:
    continue
latest_team_players.reset_index(inplace = True, drop = True)
latest_team_players.to_csv("output.csv", index=False)
latest_team_players.head()

,team_name,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11
0,KRC Genk,Marco Bizot,Timothy Castagne,Sebastien Dewaest,Christian Kabasele,Jere Uronen,Alejandro Pozuelo,Ruslan Malinovsky,"Onyinye Ndidi,26",Neeskens Kebano,Mbwana Samata,Leon Bailey
1,Beerschot AC,Stefan Deloose,Stefano Marzo,Maxime Chanot,Maeel Lepicier,Giel Deferm,Wim De Decker,Kennedy Ugoala,Joey Suk,Alpaslan Ozturk,Boldiszar Bodor,Roei Dayan
2,SV Zulte-Waregem,Kenny Steppe,Henrik Dalsgaard,Christophe Lepoint,Abdou Diallo,Bryan Verboom,Steve De Ridder,Onur Kaya,Karim Essikal,Alesandro Cordaro,Mbaye Leye,Mame Baba Thiam
3,Sporting Lokeren,Davino Verhulst,Giorgos Galitsios,Mijat Maric,Joao Carlos,Denis Odoi,Killan Overmeire,Marko Miric,Mehdi Terki,Jaja,Ayanda Patosi,Hamdi Harbaoui
4,KSV Cercle Brugge,Olivier Werner,Hans Cornelis,Noe Dussenne,Pierre Bourdin,Bart Buysse,Faris Haroun,Stipe Bacelic-Grgic,Mathieu Maertens,Kristof D'Haene,Junior Kabananga,Stephen Buyl
...,...,...,...,...,...,...,...,...,...,...,...,...
283,FC St. Gallen,Marcel Herzog,Silvan Hefti,Roy Gelmi,Martin Angha,Florent Hanin,Mario Mutsch,Marco Aratore,Danijel Aleksic,Gianluca Gaudino,Steven Lang,Albert Bunjaku
284,FC Thun,Francesco Ruberto,Kevin Bigler,Fulvio Sulmoni,Marco Buerki,Stefan Glarner,Gonzalo Zarate,Dennis Hediger,Michael Siegfried,Enrico Schirinzi,Simone Rapp,Ridge Munsy
285,Servette FC,Joao Barroca,"Issaga Diallo,28",Christopher Routis,Jerome Schneider,Francois Moubandje,Tibert Pont,Alexandre Pasche,Omar Kossoko,Antonio de Azevedo,Geoffrey Treand,Silva de Souza Eudis
286,FC Lausanne-Sports,Antonio Signori Dominique,Yaya Banana,Miha Mevlja,Jerome Sonnerat,Abdelouahed Chakhsi,Patrick Ekeng Ekeng,Yoric Ravet,Mickael Facchinetti,Yannis Tafer,Pascal Feindouno,Matar Coly
